# Connor Doman

## Research question/interests

Briefly describe your research question or interests here.

In [ ]:
import pandas as pd

df = pd.read_csv('../data/raw/Bike-Sharing-Dataset/hour.csv')

display(df.sort_values('weekday'))

# Milestone 3

## Task 2 (Which is Basically Task 1)

### 1. Load Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns

# Understanding variables
hour_df = pd.read_csv('../data/raw/Bike-Sharing-Dataset/hour.csv')
display(hour_df.shape)
display(hour_df.nunique(axis=0))
display(hour_df.head())
display(hour_df.describe())
display(hour_df.columns)

### 2. Clean Data

#### Drop Columns With More Than 40% Null Values

In [ ]:

NA_cols = hour_df.isna().sum()

def na_filter(na, threshold = 0.4):
    """Filter columns with more than 40% null values"""
    col_pass = []
    for i in na.keys():
        if na[i]/hour_df.shape[0] < threshold:
            col_pass.append(i)
    return col_pass

hour_df = hour_df[na_filter(NA_cols)]
display(hour_df.columns)


#### Drop Columns `yr`, `mnth`, `atemp`, `windspeed`, and `instant`

In [ ]:
# drop columns
for c in ['instant', 'mnth', 'yr', 'atemp', 'windspeed']:
    if c not in hour_df.columns:
        continue
    hour_df = hour_df.drop(c, axis=1)
display(hour_df.head())

#### Drop Rows That are Null Values

In [ ]:
# Drop null Values -> there are none
hour_df = hour_df.dropna(axis=0)
display(hour_df.shape)
display(hour_df.describe())

### 3. Process Data

In [ ]:
# No processing necesssary outside of cleaning

### 4. Wrangle Data

#### Reorder columns to be more relevant

In [ ]:
new_col_order = ['dteday', 'hr', 'cnt', 'casual', 'registered', 'season', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'hum']
display(hour_df[new_col_order].head())

## Task 3

In [ ]:
import pandas as pd

hours = pd.read_csv('../data/raw/Bike-Sharing-Dataset/hour.csv')

# Method chaining

new_col_order = ['date', 'hour', 'count', 'casual', 'reg', 'season', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'hum']

hours = (hours.drop(['instant', 'mnth', 'yr', 'atemp', 'windspeed'], axis=1)
         .dropna(axis=0)
         .rename(columns={"dteday": "date", "hr": "hour", "cnt": "count", "registered": "reg"})
         .sort_values("count", ascending=False))[new_col_order]

display(hours.head())

### Wrap Method Chain in a Function

In [ ]:
def load_and_process(url_or_path_to_csv_file):
    # Method Chain 1 (Load data and deal with missing data)
    
    new_col_order = ['date', 'hour', 'count', 'casual', 'reg', 'season', 'holiday', 'weekday', 'workingday', 'weather', 'temp', 'humidity']
    
    df1 = (
        pd.read_csv(url_or_path_to_csv_file)[new_col_order]
        .drop(['instant', 'mnth', 'yr', 'atemp', 'windspeed'], axis=1)
        .dropna(axis=0)
        .rename(columns={"dteday": "date", "hr": "hour", "cnt": "count", "hum": "humidity"})
        .sort_values("count", ascending=False)
    )
    
    return df1

### Import from External File

In [ ]:
import project_functions1 as pf
df = pf.load_and_process('../data/raw/Bike-Sharing-Dataset/hour.csv')
display(df.head())

## Task 4

- What days of the week have the most bike rentals?
- What holidays have the most bike rentals?
- What weather conditions have the most bike rentals?

In [ ]:
import seaborn as sns

df_to_analyze = pf.load_and_process('../data/raw/Bike-Sharing-Dataset/hour.csv')

# SELECT AVG(count) AS avgCount FROM df_to_analyze GROUP BY weekday
day_of_the_week = df_to_analyze.loc[:, ['count', 'weekday']].groupby(['weekday'], as_index=False).mean().round().sort_values('count', ascending=False)

# display(day_of_the_week)

ax = sns.barplot(data=day_of_the_week, x='weekday', y='count')
ax.set_title('Average Count of Bikes Rented by Day of the Week')
ax.set(xlabel='Day of the Week', ylabel='Average Rentals')
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")


It seems that the days of the week with the most bike rentals are 4 (Friday) and 5 (Saturday). These make sense, as these are the days that people are most likely to be off work later in the day and have time to ride bikes.

This begs another question, which time of day has the most bike rentals?

In [ ]:
# SELECT AVG(count) AS avgCount FROM df_to_analyze GROUP BY hour
hour_of_the_day = df_to_analyze.loc[:, ['count', 'hour']].groupby(['hour'], as_index=False).mean().round().sort_values('count', ascending=False)

# display(hour_of_the_day)

ax = sns.barplot(data=hour_of_the_day, x='hour', y='count')
ax.set_title("Average Rentals by Hour of the Day")
ax.set(xlabel='Time', ylabel='Average Rentals')
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")


It seems that 17:00h, or 5pm, has the most rentals on any given day. This also makes sense, as it is a rush hour commuter time. The next highest hours are 6pm and 8am, which are also rush hour times.

What about holidays?

In [ ]:
# SELECT SUM(count) AS totalCount FROM df_to_analyze WHERE holiday = 1 GROUP BY date HAVING ORDER BY totalCount DESC;
holiday = df_to_analyze[['date', 'count']].loc[df['holiday'] == 1].groupby(['date',], as_index=False).sum().sort_values('count', ascending=False)
#.groupby(['holiday'], as_index=False).mean().round().sort_values('count', ascending=False)

ax = sns.barplot(data=holiday, x='date', y='count')
ax.set_title("Total Count of Bikes Rented on Holidays")
ax.set(xlabel='Date', ylabel='Total Count')
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")

The single day with the most rentals was July 4, 2012, which makes sense. Surprisingly, the days with next most rentals were Easter 2012 and Veteran's Day 2012. While Veteran's Day was surprising, it makes sense as there are parades and processions and a lot of outdoor gatherings. Easter makes less sense outside of the fact that it is a day many people have off work.

In [ ]:
# SELECT AVG(count) AS avgCount FROM df_to_analyze WHERE date LIKE '%-11-%' GROUP BY date;
# november = df_to_analyze[['date', 'count']].loc[df['date'].str.contains('2012-11-')].groupby(['date',], as_index=False).mean().round().sort_values('count', ascending=False).mean().round()
# november = df_to_analyze.groupby(pd.PeriodIndex(df['date'], freq="D"))['count'].sum().mean()

display(f"Average rentals in November 2012: {pf.get_avg_from_month(df_to_analyze, 11, year=2012)}")
display(f"Rentals on November 12, 2012: {pf.get_avg_from_month(df_to_analyze, 11, day=12, year=2012)}")

We can see that the rentals on Veteran's Day are still over 20% higher than the average rental in November that year, which indicates the circumstance of Veteran's Day was a major factor in the high rentals. This day was a Monday, too, which removes the implication that it was a weekend day or people were out late.

In [ ]:
# Export to CSV
import project_functions1 as pf1
import pandas as pd
df = pf1.load_and_process('../data/raw/Bike-Sharing-Dataset/hour.csv')
df.to_csv('../data/processed/connor_hour.csv', index=False)

# print(pd.read_csv('../data/raw/Bike-Sharing-Dataset/hour.csv').columns)
display(df)

,date,hour,count,casual,registered,season,holiday,weekday,workingday,weather,temp,humidity
14773,2012-09-12,18:00h,977,91,886,3,0,Wednesday,1,1,0.66,0.44
14964,2012-09-20,17:00h,976,91,885,3,0,Thursday,1,1,0.64,0.50
14748,2012-09-11,17:00h,970,168,802,3,0,Tuesday,1,1,0.70,0.28
14725,2012-09-10,18:00h,968,111,857,3,0,Monday,1,1,0.62,0.35
15084,2012-09-25,17:00h,967,107,860,4,0,Tuesday,1,1,0.66,0.39
...,...,...,...,...,...,...,...,...,...,...,...,...
11304,2012-04-21,5:00h,1,0,1,2,0,Saturday,0,1,0.50,0.82
435,2011-01-20,4:00h,1,0,1,1,0,Thursday,1,1,0.26,0.56
434,2011-01-20,3:00h,1,0,1,1,0,Thursday,1,1,0.26,0.56
1041,2011-02-16,3:00h,1,0,1,1,0,Wednesday,1,2,0.20,0.47
